# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [63]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [64]:
weather_df = pd.read_csv(
    "../output_data/cities.csv", dtype="object", encoding="utf-8")
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country Date
0,avarua,-21.2078,-159.775,25.03,94,20,3.09,1637923155
1,lompoc,34.6391,-120.4579,14.55,76,1,4.12,1637923155
2,rikitea,-23.1203,-134.9692,23.72,79,97,6.04,1637923156
3,busselton,-33.65,115.3333,26.16,60,0,6.34,1637923156
4,qaanaaq,77.484,-69.3632,-14.05,85,100,1.59,1637923156
...,...,...,...,...,...,...,...,...
558,batticaloa,7.7102,81.6924,27.82,79,98,3.77,1637923318
559,roma,43.2128,-75.4557,6.18,96,90,1.54,1637923318
560,aleksandrov gay,50.147,48.5704,2.38,62,11,5.28,1637923318
561,laguna de perlas,12.3429,-83.6712,22.88,95,95,3.79,1637923319


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [65]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [66]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lng"]].astype(float)


humidity_data = weather_df["Humidity"].astype(float)

In [95]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_data, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [68]:
#A max temperature lower than 26 degrees but higher than 20 .
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

weather_df['Wind Speed']= pd.to_numeric(weather_df['Wind Speed'])
weather_df['Max Temp'] = pd.to_numeric(weather_df['Max Temp'])
weather_df['Cloudiness'] = pd.to_numeric(weather_df['Cloudiness'])

city_weather_ideal_df =  weather_df.loc[((weather_df['Max Temp']<26) & (weather_df['Max Temp']>20))&(weather_df['Wind Speed']<10)&(weather_df['Cloudiness']==0)]

city_weather_ideal_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country Date
84,namibe,-15.1961,12.1522,25.35,57,0,6.17,1637923178
188,yulara,-25.2406,130.9889,25.91,31,0,5.66,1637923209
328,san rafael,-34.6177,-68.3301,21.92,30,0,2.47,1637923251
385,santa luzia,-19.7697,-43.8514,22.60,64,0,1.54,1637923267
430,turayf,31.6725,38.6637,20.14,45,0,0.00,1637923280
434,beloha,-25.1667,45.05,25.39,36,0,7.42,1637923282
495,dadri,28.5667,77.55,25.12,25,0,0.29,1637923299
512,santiago del estero,-27.7951,-64.2615,20.45,45,0,3.05,1637923305
520,mauganj,24.6833,81.8833,22.89,29,0,2.46,1637923307


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [91]:


hotel_df =pd.DataFrame(columns = ["Hotel Name","City","Country","Lat","Lng"])

# use iterrows to iterate through pandas dataframe
for index, row in city_weather_ideal_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    location = row["Lat"] + "," + row["Lng"]
    target_radius = 5000
    params = {
               "location": location,  
               "radius": target_radius,
               "type": "lodging",
               "key": g_key
                
             }
   

    response = requests.get(base_url, params=params)
    response_json = response.json()
    
   # pprint(response_json)
    try:
        hotel_df = hotel_df.append({"Hotel Name":response_json["results"][0]["name"],
                                     "City":row["City"],
                                     "Country":response_json["results"][0]["plus_code"]["compound_code"],
                                     "Lat":row["Lat"],
                                     "Lng":row["Lng"]
                                    },ignore_index=True)
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        pass 

#x=(hotel_df['Country']).split(',')
#hotel_df['Country']=x[1]
#hotel_df 
#hotel_df["Country"].str.split(",",expand=True)
#hotel_df[['Address', 'Country']] = hotel_df['Country'].str.split(',', expand=True)
#y = x.str.split(",")
#hotel_df["Country"] = y[0][1]

hotel_df

Missing field/result... skipping.


,Hotel Name,City,Country,Lat,Lng
0,Hotel Chik Chik Namibe,namibe,"R525+V2 Namibe, Angola",-15.1961,12.1522
1,Desert Gardens Hotel - Ayers Rock Resort,yulara,"QX4M+GW Yulara NT, Australia",-25.2406,130.9889
2,Hotel San Rafael,san rafael,"9MPC+F8 San Rafael, Mendoza Province, Argentina",-34.6177,-68.3301
3,Pousada Chao Earth,santa luzia,"64MM+CJ Boa Esperança, Santa Luzia - MG, Brazil",-19.7697,-43.8514
4,Turaif,turayf,MM82+JC Turaif Saudi Arabia,31.6725,38.6637
5,Bhushan Kushwaha,dadri,"GGXR+WW Dadri, Uttar Pradesh, India",28.5667,77.55
6,Apartamentos Castelli 90,santiago del estero,"6P2W+33 Santiago del Estero, Santiago del Este...",-27.7951,-64.2615
7,Hotel Shagun Palace & Marriage Garden,mauganj,"MVJM+33 Mauganj, Madhya Pradesh, India",24.6833,81.8833


In [70]:
response_json["results"][0]["name"]

'Hotel Shagun Palace & Marriage Garden'

In [92]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]].astype(float)

hotel_info

#type(locations["Lat"])


['\n<dl>\n<dt>Name</dt><dd>Hotel Chik Chik Namibe</dd>\n<dt>City</dt><dd>namibe</dd>\n<dt>Country</dt><dd>R525+V2 Namibe, Angola</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Desert Gardens Hotel - Ayers Rock Resort</dd>\n<dt>City</dt><dd>yulara</dd>\n<dt>Country</dt><dd>QX4M+GW Yulara NT, Australia</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel San Rafael</dd>\n<dt>City</dt><dd>san rafael</dd>\n<dt>Country</dt><dd>9MPC+F8 San Rafael, Mendoza Province, Argentina</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Pousada Chao Earth</dd>\n<dt>City</dt><dd>santa luzia</dd>\n<dt>Country</dt><dd>64MM+CJ Boa Esperança, Santa Luzia - MG, Brazil</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Turaif</dd>\n<dt>City</dt><dd>turayf</dd>\n<dt>Country</dt><dd>MM82+JC Turaif Saudi Arabia</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Bhushan Kushwaha</dd>\n<dt>City</dt><dd>dadri</dd>\n<dt>Country</dt><dd>GGXR+WW Dadri, Uttar Pradesh, India</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Apartamentos Castelli 90</dd>\n<dt>

In [97]:
#markers = gmaps.marker_layer(hotel_locations)
hotel_layer = gmaps.marker_layer(
            hotel_locations, info_box_content=hotel_info)

fig = gmaps.figure()
fig.add_layer(heat_layer)
# Assign the marker layer to a variable
# Add the layer to the map

fig.add_layer(hotel_layer)
fig




Figure(layout=FigureLayout(height='420px'))